In [2]:
import numpy as np 
import pandas as pd 
from sklearn.model_selection import KFold

train = pd.read_csv('../input/commonlitreadabilityprize/train.csv')

In [3]:
kf = KFold(n_splits=3, random_state=1001,shuffle=True)
for i, (train_index, val_index) in enumerate(kf.split(train)):
    trn= train.iloc[train_index].reset_index()
    val= train.iloc[val_index].reset_index()
    
trn = trn.drop(columns=['index'])
val = val.drop(columns=['index'])

val.to_csv('sub_val.csv',index=False)
trn.to_csv('sub_train.csv',index=False)

In [4]:
# !rm -r ./autox
!git clone https://github.com/4paradigm/autox.git
!pip install ./autox

Cloning into 'autox'...
remote: Enumerating objects: 2699, done.
remote: Counting objects: 100% (612/612), done.
remote: Compressing objects: 100% (153/153), done.
remote: Total 2699 (delta 479), reused 570 (delta 452), pack-reused 2087
Receiving objects: 100% (2699/2699), 11.87 MiB | 26.03 MiB/s, done.
Resolving deltas: 100% (1723/1723), done.
Processing ./autox
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 948.9/948.9 KB 51.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.4/325.4 KB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 17.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 62.0 MB/s eta 0:00:0000:0100:01
  Created wheel for automl-x: filename=automl_x-0

In [5]:
from autox.autox_nlp import NLP_feature
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

In [6]:
df_train = pd.read_csv('sub_train.csv')
df_test = pd.read_csv('sub_val.csv')

In [7]:
use_Toknizer=True
emb_mode = 'Bert'# TFIDF / Word2Vec / Glove / FastText / Bert
encode_mode = 'supervise' # unsupervise / supervise
text_columns_name = ['excerpt']
target_column = df_train['target']
candidate_labels=None

In [8]:
nlp = NLP_feature()
nlp.do_mlm = True
nlp.mlm_epochs=3
nlp.model_name = 'microsoft/deberta-v3-base'
nlp.emb_size=100
nlp.n_clusters=20
df = nlp.fit(df_train,
             text_columns_name,
             use_Toknizer,
             emb_mode,
             encode_mode,
             target_column,
             candidate_labels)

Fitting column: excerpt tokenizer


Downloading:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/579 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.35M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

Some weights of the model checkpoint at microsoft/deberta-v3-base were not used when initializing DebertaV2ForMaskedLM: ['mask_predictions.dense.weight', 'lm_predictions.lm_head.bias', 'mask_predictions.dense.bias', 'mask_predictions.LayerNorm.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'deberta.embeddings.position_embeddings.weight', 'mask_predictions.classifier.bias', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.dense.bias', 'mask_predictions.classifier.weight', 'mask_predictions.LayerNorm.weight']
- This IS expected if you are initializing DebertaV2ForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2ForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model

Step,Training Loss
500,5.744500




Training completed. Do not forget to share your model on huggingface.co/models =)


Saving model checkpoint to ./excerpt_transformer
Configuration saved in ./excerpt_transformer/config.json
Model weights saved in ./excerpt_transformer/pytorch_model.bin
loading configuration file ./excerpt_transformer/config.json
Model config DebertaV2Config {
  "_name_or_path": "./excerpt_transformer",
  "architectures": [
    "DebertaV2ForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-07,
  "max_position_embeddings": 512,
  "max_relative_positions": -1,
  "model_type": "deberta-v2",
  "norm_rel_ebd": "layer_norm",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_hidden_states": true,
  "pad_token_id": 0,
  "pooler_dropout": 0,
  "pooler_hidden_act": "gelu",
  "pooler_hidden_size": 768,
  "pos_att_type": [
    "p2c",
    "

Fitting column: excerpt encoder


Some weights of the model checkpoint at ./excerpt_transformer were not used when initializing DebertaV2Model: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of DebertaV2Model were initialized from the model checkpoint at ./excerpt_transformer.
If your task is similar to the task the model of the checkpoi

In [9]:
for column in df.columns:
    df_train[column] = df[column]
df_train = df_train.drop(columns=text_columns_name)

In [10]:
test = nlp.transform(df_test)

loading configuration file ./excerpt_transformer/config.json
Model config DebertaV2Config {
  "_name_or_path": "./excerpt_transformer",
  "architectures": [
    "DebertaV2ForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-07,
  "max_position_embeddings": 512,
  "max_relative_positions": -1,
  "model_type": "deberta-v2",
  "norm_rel_ebd": "layer_norm",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_hidden_states": true,
  "pad_token_id": 0,
  "pooler_dropout": 0,
  "pooler_hidden_act": "gelu",
  "pooler_hidden_size": 768,
  "pos_att_type": [
    "p2c",
    "c2p"
  ],
  "position_biased_input": false,
  "position_buckets": 256,
  "relative_attention": true,
  "share_att_key": true,
  "torch_dtype": "float32",
  "transformers_version": "4.18.0",
  "type_vocab_size": 0,
  "vocab_size": 128100
}

loading weights

Transforming column: excerpt


Some weights of the model checkpoint at ./excerpt_transformer were not used when initializing DebertaV2Model: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of DebertaV2Model were initialized from the model checkpoint at ./excerpt_transformer.
If your task is similar to the task the model of the checkpoi

In [11]:
for column in test.columns:
    df_test[column] = test[column]
df_test = df_test.drop(columns=text_columns_name)

In [12]:
df_train.to_csv(f'{emb_mode}_{encode_mode}_autox_trn.csv',index=False)
df_test.to_csv(f'{emb_mode}_{encode_mode}_autox_val.csv',index=False)

In [13]:
df_val=pd.read_csv(f'{emb_mode}_{encode_mode}_autox_val.csv').drop(columns=['target'])
df_val.to_csv(f'{emb_mode}_{encode_mode}_autox_tst.csv',index=False)

In [14]:
from autox import AutoX

path = f'.' 
autox = AutoX(target = 'target', train_name = f'{emb_mode}_{encode_mode}_autox_trn.csv', test_name = f'{emb_mode}_{encode_mode}_autox_tst.csv',  id = [], path = path)
sub = autox.get_submit()

   INFO ->  [+] read Bert_supervise_autox_trn.csv
   INFO ->  Memory usage of dataframe is 0.09 MB
   INFO ->  Memory usage after optimization is: 0.12 MB
   INFO ->  Decreased by -35.3%
   INFO ->  table = Bert_supervise_autox_trn.csv, shape = (1890, 6)
   INFO ->  [+] read Bert_supervise_autox_val.csv
   INFO ->  Memory usage of dataframe is 0.04 MB
   INFO ->  Memory usage after optimization is: 0.06 MB
   INFO ->  Decreased by -35.5%
   INFO ->  table = Bert_supervise_autox_val.csv, shape = (944, 6)
   INFO ->  [+] read Bert_supervise_autox_tst.csv
   INFO ->  Memory usage of dataframe is 0.04 MB
   INFO ->  Memory usage after optimization is: 0.06 MB
   INFO ->  Decreased by -57.5%
   INFO ->  table = Bert_supervise_autox_tst.csv, shape = (944, 5)
   INFO ->  [+] read sub_train.csv
   INFO ->  Memory usage of dataframe is 0.09 MB
   INFO ->  Memory usage after optimization is: 0.19 MB
   INFO ->  Decreased by -124.7%
   INFO ->  table = sub_train.csv, shape = (1890, 6)
   INFO -> 

(1890, 98) (944, 98)



  0%|          | 0/1 [00:00<?, ?it/s]

Training on model 1 - fold 1
Training until validation scores don't improve for 150 rounds
[20]	training's auc: 0.65212	valid_1's auc: 0.655716
[40]	training's auc: 0.65287	valid_1's auc: 0.656997
[60]	training's auc: 0.652993	valid_1's auc: 0.657898
[80]	training's auc: 0.653785	valid_1's auc: 0.658942
[100]	training's auc: 0.65408	valid_1's auc: 0.65899
[120]	training's auc: 0.654514	valid_1's auc: 0.659132
[140]	training's auc: 0.655216	valid_1's auc: 0.65899
[160]	training's auc: 0.663799	valid_1's auc: 0.653107
[180]	training's auc: 0.664395	valid_1's auc: 0.655527
[200]	training's auc: 0.664483	valid_1's auc: 0.655384
[220]	training's auc: 0.664558	valid_1's auc: 0.655432
[240]	training's auc: 0.66461	valid_1's auc: 0.655337
[260]	training's auc: 0.664622	valid_1's auc: 0.655337
Early stopping, best iteration is:
[125]	training's auc: 0.655031	valid_1's auc: 0.659274
Model 1 - Fold 1 finished in 0:00:01.407168
Training on model 1 - fold 2
Training until validation scores don't im


100%|██████████| 1/1 [00:09<00:00,  9.22s/it]
   INFO ->  featureNlp ops: [['url_legal']]
   INFO ->  feature engineer: Count


...

Training on fold 1
Training until validation scores don't improve for 150 rounds
[100]	training's rmse: 0.632409	valid_1's rmse: 0.66319
[200]	training's rmse: 0.522142	valid_1's rmse: 0.603235
[300]	training's rmse: 0.488352	valid_1's rmse: 0.596521
[400]	training's rmse: 0.472772	valid_1's rmse: 0.597198
Early stopping, best iteration is:
[291]	training's rmse: 0.490233	valid_1's rmse: 0.596443
MSE: 0.35574478133300824
Fold 1 finished in 0:00:05.787609
Training on fold 2
Training until validation scores don't improve for 150 rounds
[100]	training's rmse: 0.632321	valid_1's rmse: 0.674792
[200]	training's rmse: 0.529629	valid_1's rmse: 0.580651
[300]	training's rmse: 0.497538	valid_1's rmse: 0.560305
[400]	training's rmse: 0.48264	valid_1's rmse: 0.555218
[500]	training's rmse: 0.474282	valid_1's rmse: 0.554382
[600]	training's rmse: 0.467293	valid_1's rmse: 0.554919
Early stopping, best iteration is:
[487]	training's rmse: 0.47509	valid_1's rmse: 0.554308
MSE: 0.307257663142463
Fold 

   INFO ->  feature importance
   INFO ->                                    feature  fold_1  fold_2  fold_3  fold_4  \
0                          standard_error    1328    1545    1459    1594   
1                    excerpt_meta_feature    1098    1460    1293    1401   
2     license__excerpt_meta_feature__rank     680     770     716     799   
3           license__standard_error__rank     503     672     646     760   
4                           url_legal_nlp     237     316     272     379   
..                                    ...     ...     ...     ...     ...   
60     license__excerpt_meta_feature__max       0       4       5       6   
61     license__excerpt_meta_feature__std       0       2       2       1   
62           license__standard_error__std       0       1       2       1   
63     license__excerpt_meta_feature__min       0       1       0       2   
64  license__excerpt_meta_feature__median       0       0       0       2   

    fold_5  average  
0     1502

Early stopping, best iteration is:
[432]	training's rmse: 0.48697	valid_1's rmse: 0.528675
MSE: 0.279497485857031
Fold 5 finished in 0:00:07.519047
Training on fold 1
[13:59:52] WARNING: ../src/learner.cc:627: 
Parameters: { "verbose_eval" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	validation_0-rmse:1.77590
[100]	validation_0-rmse:0.96613
[200]	validation_0-rmse:0.71288
[300]	validation_0-rmse:0.63743
[400]	validation_0-rmse:0.61348
[500]	validation_0-rmse:0.60279
[600]	validation_0-rmse:0.59838
[700]	validation_0-rmse:0.59572
[800]	validation_0-rmse:0.59491
[900]	validation_0-rmse:0.59451
[1000]	validation_0-rmse:0.59424
[1100]	validation_0-rmse:0.59413
[1200]	validation_0-rmse:0.59366
[1300]	validation_0-rmse:0.59357
[1400]	validation_0-

   INFO ->  Average KFold RMSE: 0.37942707538604736


MSE: 0.3553660213947296
Fold 10 finished in 0:00:06.307361


In [15]:
val = pd.read_csv(f'sub_val.csv')
from sklearn.metrics import mean_squared_error
RMSE = np.sqrt(mean_squared_error(val['target'], sub['target']))

In [16]:
RMSE

0.5970512889627345